In [ ]:
import pandas as pd
import numpy as np
import os
import gzip
import pickle
import re
import copy
from tqdm import tqdm
import time
import json
from datetime import datetime, timedelta
# pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 100)

In [ ]:
data = pd.read_csv('../5_processing_extracted_data/hardware_instances_with_efficiency.csv') # date,row_index,hardware_name,TH/J

# Convert the 'date' column to a datetime format
data['date'] = pd.to_datetime(data['date'])

# keep only data from 2011 onwards
data = data[data['date'] >= '2011-01-02']

# Create a 'month' column
data['month'] = (data['date']).dt.to_period('M')

# Group by 'month' and calculate the mean of 'TH/J'
quarterly_data = data.groupby('month')['TH/J'].mean().reset_index()

# Convert 'month' back to datetime (first day of the month)
quarterly_data['month'] = quarterly_data['month'].dt.to_timestamp()


# monthly_efficiency contains:
# Date,ln(P_max),Data monthly average,Model ln(P_eff)
# 2011-04-01,-12.485786548332777,1.6612383008356546e-06,1.5657608695648479e-06
monthly_efficiency = quarterly_data

In [ ]:
# hashrate,coins_per_block,date
# 4318540.0,50.0,2009-01-01
BitcoinData = pd.read_csv('./BitcoinData/BitcoinData.csv')
BitcoinData['date'] = pd.to_datetime(BitcoinData['date'])

In [ ]:
# Start,End,Open,High,Low,Close,Volume,Market Cap
# 2024-03-08,2024-03-09,66871.2,69451.15,66398.03,68202.87,71117263448.78049,1329444482209.993
price = pd.read_csv('BitcoinData/pricehistory/price_full.csv') 



df1 = pd.DataFrame(columns=['date', 'price'])
df3 = pd.DataFrame(columns=['date', 'efficiency'])


df1['date'] = pd.to_datetime(price['Start'])
df1['price'] = price['Close']

df3['date'] = pd.to_datetime(monthly_efficiency['month'], format='%Y-%m-%d')
df3['efficiency'] = monthly_efficiency['TH/J']*1e12

joined = pd.merge(df1, BitcoinData, on='date', how='inner')
joined = pd.merge(joined, df3, on='date', how='inner')
joined = joined.dropna()

In [ ]:
pd.set_option('display.max_rows', None)
joined.head(3)

In [ ]:
df = joined

In [ ]:
df['date'] = pd.to_datetime(df['date']).dt.date
df.head(3)

In [ ]:
max_efficiency_df = pd.read_csv('../../hardwarelist/pmaxv3.csv', usecols=['date', 'max_efficiency'])
max_efficiency_df['date'] = pd.to_datetime(max_efficiency_df['date']).dt.date
# Create a 'month' column
max_efficiency_df['month'] = pd.to_datetime(max_efficiency_df['date']).dt.to_period('M')
# Group by 'month' and calculate the max of 'TH/J'
max_efficiency_df = max_efficiency_df.groupby('month')['max_efficiency'].max().reset_index()
# Convert 'month' back to datetime (first day of the month)
max_efficiency_df['date'] = max_efficiency_df['month'].dt.to_timestamp()
max_efficiency_df.drop(columns=["month"], inplace=True)
max_efficiency_df.tail(5)

In [ ]:
# Merge with max efficiency
df['date'] = df['date'].astype(str)
max_efficiency_df['date'] = max_efficiency_df['date'].astype(str)
df = pd.merge(df, max_efficiency_df, on='date', how='left')

In [ ]:
# Convert dates to end of the month
df['date'] = pd.to_datetime(df['date']) + pd.offsets.MonthEnd(0)

In [ ]:
df.to_csv('monthly_stuff.csv', index=False)